# <center> Social Platform Emulator </center>

In [1]:
import sys
import util
import random
import numpy as np
from time import time
from operator import itemgetter

In [2]:
dataset = 'weibo_rtid'

data_path, RTU, truegraph = util.load_data(dataset)
out_path = "../PsiResults/{}/Psi_emul.txt".format(dataset)

Wall is finite for all users. We apply FIFO replacement principle. The Newsfeed is not needed, because we have no information on the policy of the OSP or the user policy to choose among many. We know: 1) which post is tweeted-retweeted and the time-stamp, and the tweet origin.

For each user we create a Wall list of max size $K$.

In [3]:
K = 1 # Wall, Newsfeed list size
Wall = dict() # les walls

If we have RTids and not RTusers we create the `Author` dict.

In [4]:
if not RTU:
    
    # create author dict
    Author = util.get_authors(data_path)
        
    # check length and other stuff
    print("Length Author dict : ", len(Author))
    print("74400 in Author : ", 74400 in Author)

Length Author dict :  33540167
74400 in Author :  True


We introduce a disctionary `Q`, which contains many dictionries, each related to a userid. $Q[userid][auth-out]$ saves the time periods that a post from $[auth-out]$ stays on the Wall of $[userid]$.

In [5]:
Q = dict()
FirstP = dict()

**Note:** Are we happy with Q as matrix? or list of lists? or txt? or dictionary? **A: Dictionary of Dictionaries**

**Important! In the case of RTids :**

if the tweet is original then the person who posted is the author.
    else it is a retweet. If the original post was observed and saved in "Authors",
   we know the author.
   else the retweet has origin outside the dataset. In this case we assume that
   the first retweeter is the original author.

In [6]:
Amphitrions = dict() # for each user, gathers the users present on his wall in case of a tie
Time0 = None # time of the first event
timestamp_old = 0 # for time tracking
count_ties = 0 # used in case of multiple posts on the same wall at the same time
 

# iterate
start = time()
for i,lign in enumerate(open(data_path)):
    
    # print state
    if i%10000==0:
        sys.stdout.flush()
        sys.stdout.write("line {}... elapsed time {}\r".format(i, time()-start))

    # split
    lign = lign.split()
    if RTU:
        tweetid, tstamp, userid, rtu = int(lign[0]), int(lign[1]), int(lign[2]), int(lign[3])
    else:
        tweetid, tstamp, userid, rtid = int(lign[0]), int(lign[1]), int(lign[2]), int(lign[3])  
    
    # time tracking
    if Time0 == None:
        Time0 = tstamp
    if tstamp == timestamp_old:
        count_ties+=1
    timestamp_old = tstamp
    
    # add userid to Amphitrions
    if userid not in Amphitrions:
        Amphitrions[userid] = dict()

    # rtu treatment
    if RTU:
        if rtu == -1:
            auth = userid
        else:
            auth = rtu
    
    # else rtid treatment
    else:
        if rtid == -1:
            auth = userid
        elif rtid in Author:
            auth = Author[rtid] # it is the tweet origin ID
        else:
            auth = userid
        
    # add userid to Wall and Q dicts
    if userid not in Wall:
        Wall[userid] = []
        Q[userid] = dict()
        
    # is it the user's first post ? if so update FirstP dict
    if len(Wall[userid])==0:
        FirstP[userid] = tstamp
    
    # add new post to userid's wall
    Wall[userid].append((tweetid,tstamp,auth))
    
    # if there is already a post on the wall
    if len(Wall[userid]) > K:
        
        # update auth_out and time_wall
        auth_out = Wall[userid][0][2]
        time_wall = tstamp-Wall[userid][0][1]

        # add auth_out to Q[userid]
        if auth_out not in Q[userid]:
            Q[userid][auth_out] = 0

        # update AmphiAmphitrions[userid] with auth_out
        if auth_out not in Amphitrions[userid]:
            Amphitrions[userid][auth_out] = 0 
        Amphitrions[userid][auth_out] += 1
          
        # if no tie (or end of a tie)
        if time_wall != 0:
            Ndt = 0 # reinit number of ties
            for author in Amphitrions[userid]: # count ties
                Ndt += Amphitrions[userid][author]
            for author in Amphitrions[userid]: # update Q[userid]
                Q[userid][author] += time_wall / Ndt * Amphitrions[userid][author]
            Amphitrions[userid] = dict() # reinit Amphitrions[userid]
            
        # keep the most recent post on wall (<=> random eviction because K=1)
        Wall[userid] = Wall[userid][1:]
        assert( len(Wall[userid]) == K )

        
print("\nNb ties : ", count_ties)

line 33540000... elapsed time 643.77880883216864
Nb ties :  12088613


**Note1:** EndPoint of simulation is the time of last arrival.

In [7]:
print("Init time : ", Time0)
EndP = tstamp
print("EndPoint of simulation : ", EndP)

if not RTU:
    print("Length Author dict : ", len(Author))

Init time :  1251453071
EndPoint of simulation :  1356506843
Length Author dict :  33540167


**Note2:** Post-processing: Treat extra users with Wall not empty at the end of the simulation. In this case, we consider that the post on their wall was also there when the simulation started and until the first post of the concerned user.

In [8]:
# iterate over all walls
for u in Wall:
    
    # if wall not empty
    if len(Wall[u]) > 0:
        
        # get auth_out and time on wall
        auth_out = Wall[u][0][2]
        time_wall = EndP-Wall[u][0][1]
        
        # update Q[u] with auth_out
        if u not in Q:
            Q[u] = dict()
        if auth_out not in Q[u]:
            Q[u][auth_out] = 0

        # update Amphitrions[u]
        if auth_out not in Amphitrions[u]:
            Amphitrions[u][auth_out] = 0 
        Amphitrions[u][auth_out] += 1
        
        # if no tie (or end of a tie)
        if time_wall != 0:
            Ndt = 0 # reinit nb ties
            for author in Amphitrions[u]: # count ties
                Ndt += Amphitrions[u][author]
            for author in Amphitrions[u]: # update Q[u] with AmphitAmphitrions
                Q[u][author] += ( time_wall + FirstP[u] - Time0 ) / Ndt * Amphitrions[u][author]
            Amphitrions[u] = dict() # reinit Amphitrions[u]

In [9]:
user = random.choice(list(Q.keys()))
print("Q of user {} : {}".format(user, Q[user]))

Q of user 728108 : {1038127: 6392578.0, 1610757: 5086558.0, 836276: 1293104.0, 1339318: 6816994.0, 884856: 2161396.0, 7206: 1677807.0, 148795: 2226.0, 236648: 129473.0, 608659: 874002.0, 820273: 2716215.0, 861288: 368.0, 216814: 172009.0, 219985: 1302361.0, 439305: 1117437.0, 535188: 75311244.0}


**Estimated Qs**
We now compoute the end values for `Q` by dividing each one with the total duration of the simulation.

In [10]:
EstQ = dict()
nb0 = 0

for u in Q:
    
    # we don't consider users that made their first post at the last timestamp
    if EndP - FirstP[u] == 0:
        nb0 += 1
        assert len(Q[u]) == 1
    
    # otherwise we compute the final value of Q for the user u
    else:
        EstQ[u] = { j: Q[u][j]/(EndP - Time0) for j in Q[u] }

print(nb0)

0


Print length of `EstQ`.

In [11]:
N = len(EstQ)
print("Length of EstQ dict : ", N)

Length of EstQ dict :  1340816


In [12]:
print("EstQ of user {} : {}".format(user, EstQ[user]))

EstQ of user 728108 : {1038127: 0.060850532810949425, 1610757: 0.04841861366005973, 836276: 0.012308972589770503, 1339318: 0.06489052101813156, 884856: 0.020574187474201307, 7206: 0.01597093534156965, 148795: 2.1189148734231074e-05, 236648: 0.0012324450377659928, 608659: 0.008319568001803876, 820273: 0.02585547332845888, 861288: 3.5029679848144815e-06, 216814: 0.0016373424459237885, 219985: 0.012397089368671121, 439305: 0.01063680988056288, 535188: 0.7168828169254122}


**EstQ[user][leader]:** is the proportion of time that posts of "leader" is on 1-st position of "user" Wall. Hence:
$\sum_{leaders}EstQ[user][leader] = 1$.

Calculate Influence of user on his followers.

In [13]:
Influence = dict()

for follower in EstQ:
    for leader in EstQ[follower]:
        
        if leader not in Influence:
            Influence[leader] = dict()
            
        Influence[leader][follower] = EstQ[follower][leader]

In [14]:
leader = random.choice(list(EstQ[user].keys()))
print("Influence of user {} : {}".format(leader, Influence[leader]))

Influence of user 1610757 : {1486229: 0.0008040453797318196, 901604: 1.4487818676325112e-05, 44025: 0.016385047078557063, 498760: 0.00045633773150001694, 1494124: 0.0024245107543592057, 0: 0.0033567095525137355, 903607: 0.0008573799710875683, 76601: 0.0022437747404253127, 878797: 4.608116308284485e-05, 909668: 0.001455483197690417, 228755: 0.00033761757740597835, 1489700: 0.007029428700570599, 510359: 0.0017561673082999818, 485585: 0.0024450050208573186, 1501411: 0.15103630929120757, 488171: 0.004191482053590613, 1504660: 0.023056601908592107, 242513: 0.0107283344381009, 1503083: 0.0032111174456448837, 921572: 0.11618373874286018, 915119: 0.0024245107543592057, 920144: 1.9228248177514272e-06, 70015: 0.008917623633733019, 246403: 0.0016487461297439183, 86132: 0.0006120770228031413, 244538: 0.10106413884881735, 80227: 0.021777847253309477, 1483020: 0.009345033322554092, 927448: 0.003410643836758189, 84415: 0.00015875679361613022, 487027: 0.0008727720885643211, 247959: 0.00516568791075869

Compute final values for $\psi$.

In [15]:
Psi = dict()
for user in Influence:
    Psi[user] = 0
    for follower in Influence[user]:
        if follower != user:
            Psi[user]+=Influence[user][follower]
    Psi[user] = Psi[user]/(N-1)

In [16]:
print("Psi of user {} : {}".format(leader, Psi[leader]))

Psi of user 1610757 : 0.0004033753426929115


**Export sorted list of $\psi$.**

In [17]:
Psi = sorted(Psi.items(), key=itemgetter(1), reverse=True)

In [18]:
with open(out_path, 'w') as f:
    for p in Psi:
        f.write("%d %g\n"%(p[0], p[1]))

In [19]:
print(Psi[-5:])

[(300947, 0.0), (2575, 0.0), (601042, 0.0), (900104, 0.0), (722, 0.0)]
